In [1]:
import pandas as pd
df = pd.read_csv("sidd.csv")

In [2]:
df

,Unnamed: 0,NCT Number,Study Title,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,...,reason_personal_reasons,reason_ongoing,reason_serious_adverse_event,reason_withdrawal_of_consent,reason_progressive_disease,reason_other_than_specified,reason_abnormal_laboratory_value(s),reason_missing,reason_unclassified,duration
0,0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,COMPLETED,The hypothesis of this study is use of CytoSor...,NO,Acute Respiratory Distress Syndrome|Acute Lung...,DEVICE: CytoSorb Hemoperfusion,Relative IL-6 levels as a percent (%) of basel...,"Ventilator Free Days, Reduction cytokines TNF-...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.784190
1,1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,NON COMPLETED,The primary purpose of this study is to find o...,NO,Cancer|Solid Tumors|Advanced Solid Malignancies,DRUG: AZD7762|DRUG: gemcitabine,Assessment of adverse events (based on CTCAE v...,Pharmacokinetic effect of AZD7762 when adminis...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.995732
2,2,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,COMPLETED,To study the impact of 3 day exposure to atorv...,NO,Ischemia Reperfusion Injury|Cardiovascular Dis...,DRUG: atorvastatin,Annexin A 5 targeting in the non dominant then...,"workload during ischemic exercise, workload du...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.258097
3,3,NCT03296228,Comparison of Dynamic Radiographs in Determini...,COMPLETED,The purpose of this study is to identify the f...,NO,Adolescent Idiopathic Scoliosis,"RADIATION: Flexibility Radiographs (supine, su...",Investigate the flexibility equivalence of dif...,Incorporate these findings into the Lenke Clas...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.583519
4,4,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,COMPLETED,"The proposed protocol is a double-blind, place...",YES,Cocaine Dependence,DRUG: Adderall-XR and Topiramate|DRUG: Placebo,Three Weeks of Continuous Cocaine Abstinence a...,UNKNOWN_MEASURES,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.688879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257571,257571,NCT02360800,Trial Comparing Tisseel as an Adjunct to Tradi...,COMPLETED,Bleeding after redo cardiac surgery is a commo...,NO,Hemorrhage,PROCEDURE: Spray Fibrin Sealant|PROCEDURE: Tra...,Bleeding (Amount of blood (ml) collected in ch...,Allogeneic blood Sparing (Amount of Allogeneic...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.995732
257572,257572,NCT02352506,Serum MIF in Acute Kidney Injury,COMPLETED,Acute kidney injury (AKI) is a common complica...,NO,Acute Kidney Injury,OTHER: Blood draw for measurement of MIF,"Macrophage migration inhibitory factor, Up to ...","Serum creatinine, Up to 14 days",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.465908
257573,257573,NCT04996381,Feasibility of AI-based Heart Function Predict...,COMPLETED,The investigators will develop an artificial i...,NO,Chest X-ray for Clinical Evaluation,DIAGNOSTIC_TEST: Scanning Chest X-rays and per...,"Left Ventricular Ejection Fraction < 40%, Eval...",UNKNOWN_MEASURES,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
257574,257574,NCT00380640,The Efficacy of Trimethoprim in Wound Healing ...,COMPLETED,The purpose of this study is to assess the eff...,NO,Epidermolysis Bullosa,DRUG: Trimethoprim|DRUG: Trimethoprim,Percentage change of area of the wound from vi...,"Total number of blisters at each visit, At 2 m...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.564949


In [3]:
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

In [4]:
df.shape

(257576, 280)

Low Cardinality Columns : Applying Label Encoding

In [5]:
from sklearn.preprocessing import LabelEncoder

#binary categorical column (YES/NO)
df["Study Results"] = df["Study Results"].map({"YES": 1, "NO": 0})

small_cat_features = ["Study Type", "sampling_method", "Sex", "Phases", "Funder Type"]

label_encoder = LabelEncoder()
for col in small_cat_features:
    df[col] = label_encoder.fit_transform(df[col])

print("\n Label Encoding applied to small categorical features.")


 Label Encoding applied to small categorical features.


Applying MultiLabel Binarizer on `Study Design`

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

df["Study Design"] = df["Study Design"].fillna("").apply(lambda x: x.split("|"))

mlb_study_design = MultiLabelBinarizer()
study_design_encoded = pd.DataFrame(
    mlb_study_design.fit_transform(df["Study Design"]),
    columns=[f"Study_Design_{col}" for col in mlb_study_design.classes_]  # Adding Prefix
)

df = pd.concat([df.reset_index(drop=True), study_design_encoded], axis=1)
df.drop(columns=["Study Design"], inplace=True)

print("\n MultiLabel Binarization applied to 'Study Design' with prefix")



 MultiLabel Binarization applied to 'Study Design' with prefix


In [7]:
df.shape

(257576, 322)

Applying TargetEncoding on `Conditions` and `Interventions`

In [8]:
df["Study Status"].unique()

array(['COMPLETED', 'NON COMPLETED'], dtype=object)

In [9]:
import pandas as pd

# Function to apply Target Encoding while keeping rows the same
def target_encode_average(df, column, target_col):
    """
    Applies target encoding to multi-label categorical columns by:
    - Splitting the column on '|'
    - Mapping each individual value to its target-encoded value
    - Taking the average for each row (instead of exploding into multiple rows)
    """
    # Compute mean Study Status for each unique category
    target_map = df.explode(column).groupby(column)[target_col].mean()

    # Function to compute the average target encoding for a row
    def encode_row(value):
        if isinstance(value, str):
            categories = value.split('|')  # Split multi-label values
            encoded_values = [target_map.get(cat, target_map.mean()) for cat in categories]  # Map each category
            return sum(encoded_values) / len(encoded_values)  # Compute row-wise average
        return value  # If it's not a string, return as is
    
    # Apply encoding to the column
    df[column] = df[column].apply(encode_row)
    
    return df

# Ensure "Study Status" is properly encoded before target encoding
df["Study Status"] = df["Study Status"].map({"COMPLETED": 1, "NON COMPLETED": 0})

# Apply target encoding while taking the average within each row
df_target_encoded = df.copy()
df_target_encoded = target_encode_average(df_target_encoded, "Conditions", "Study Status")
df_target_encoded = target_encode_average(df_target_encoded, "Interventions", "Study Status")

# Handling unknown categories (replace with global mean)
global_mean_study_status = df["Study Status"].mean()
df_target_encoded["Conditions"].fillna(global_mean_study_status, inplace=True)
df_target_encoded["Interventions"].fillna(global_mean_study_status, inplace=True)

# Display the updated dataset
from IPython.display import display
display(df_target_encoded)


,NCT Number,Study Title,Study Status,Brief Summary,Study Results,Conditions,Interventions,Primary Outcome Measures,Secondary Outcome Measures,Sex,...,Study_Design_Primary Purpose: DIAGNOSTIC,Study_Design_Primary Purpose: ECT,Study_Design_Primary Purpose: HEALTH_SERVICES_RESEARCH,Study_Design_Primary Purpose: OTHER,Study_Design_Primary Purpose: PREVENTION,Study_Design_Primary Purpose: SCREENING,Study_Design_Primary Purpose: SUPPORTIVE_CARE,Study_Design_Primary Purpose: TREATMENT,Study_Design_Time Perspective: p,Study_Design_UNKNOWN_DESIGN
0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,1,The hypothesis of this study is use of CytoSor...,0,0.787961,1.000000,Relative IL-6 levels as a percent (%) of basel...,"Ventilator Free Days, Reduction cytokines TNF-...",0,...,0,0,0,0,0,0,0,1,0,0
1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,0,The primary purpose of this study is to find o...,0,0.871238,0.427615,Assessment of adverse events (based on CTCAE v...,Pharmacokinetic effect of AZD7762 when adminis...,0,...,0,0,0,0,0,0,0,1,0,0
2,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,1,To study the impact of 3 day exposure to atorv...,0,0.902222,0.916667,Annexin A 5 targeting in the non dominant then...,"workload during ischemic exercise, workload du...",2,...,0,0,0,0,0,0,0,1,0,0
3,NCT03296228,Comparison of Dynamic Radiographs in Determini...,1,The purpose of this study is to identify the f...,0,0.826087,0.855230,Investigate the flexibility equivalence of dif...,Incorporate these findings into the Lenke Clas...,0,...,0,0,0,0,0,0,0,0,1,0
4,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,1,"The proposed protocol is a double-blind, place...",1,0.858974,0.827615,Three Weeks of Continuous Cocaine Abstinence a...,UNKNOWN_MEASURES,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257571,NCT02360800,Trial Comparing Tisseel as an Adjunct to Tradi...,1,Bleeding after redo cardiac surgery is a commo...,0,0.900000,0.855230,Bleeding (Amount of blood (ml) collected in ch...,Allogeneic blood Sparing (Amount of Allogeneic...,0,...,0,0,0,0,0,0,0,1,0,0
257572,NCT02352506,Serum MIF in Acute Kidney Injury,1,Acute kidney injury (AKI) is a common complica...,0,0.849711,1.000000,"Macrophage migration inhibitory factor, Up to ...","Serum creatinine, Up to 14 days",0,...,0,0,0,0,0,0,0,0,1,0
257573,NCT04996381,Feasibility of AI-based Heart Function Predict...,1,The investigators will develop an artificial i...,0,1.000000,1.000000,"Left Ventricular Ejection Fraction < 40%, Eval...",UNKNOWN_MEASURES,0,...,0,0,0,0,0,0,0,0,1,0
257574,NCT00380640,The Efficacy of Trimethoprim in Wound Healing ...,1,The purpose of this study is to assess the eff...,0,0.714286,0.855230,Percentage change of area of the wound from vi...,"Total number of blisters at each visit, At 2 m...",0,...,0,0,0,0,0,0,0,1,0,0


In [21]:
df_target_encoded.shape

(257576, 321)

In [20]:
df_target_encoded["Study Status"]

0         1
1         0
2         1
3         1
4         1
         ..
257571    1
257572    1
257573    1
257574    1
257575    1
Name: Study Status, Length: 257576, dtype: int64

Applying TF-IDF to columns with `High Cardinality`

In [18]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from scipy import sparse

# Download necessary resources
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

# Function for text cleaning and preprocessing
def preprocess_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # Remove punctuation
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])  # Lemmatization & stopword removal
    return text

# Create a copy of the dataset for safe transformation
df_transformed = df_target_encoded.copy()

# Separate TF-IDF for each text column
primary_tfidf = TfidfVectorizer(max_features=2500, ngram_range=(1,1))
secondary_tfidf = TfidfVectorizer(max_features=2500, ngram_range=(1,1))

# Fit and transform each column separately
X_primary = primary_tfidf.fit_transform(df_transformed["Primary Outcome Measures"].fillna("").apply(preprocess_text))
X_secondary = secondary_tfidf.fit_transform(df_transformed["Secondary Outcome Measures"].fillna("").apply(preprocess_text))

# Get target variable
y = df_transformed["Study Status"]

# Combine features for chi-square selection
X_combined = sparse.hstack([X_primary, X_secondary])
combined_features = (
    [f"TFIDF_primaryoutcomes_{w}" for w in primary_tfidf.get_feature_names_out()] +
    [f"TFIDF_secondaryoutcomes_{w}" for w in secondary_tfidf.get_feature_names_out()]
)

# Select top features
selector = SelectKBest(chi2, k=500)
X_combined_selected = selector.fit_transform(X_combined, y)

# Get selected feature names with their source
selected_feature_names = [combined_features[i] for i in selector.get_support(indices=True)]

# Convert to DataFrame
tfidf_selected_df = pd.DataFrame(
    X_combined_selected.toarray(),
    columns=selected_feature_names
)

# Merge and clean up
df_transformed = pd.concat([df_transformed.reset_index(drop=True), tfidf_selected_df], axis=1)
df_transformed.drop(columns=["Primary Outcome Measures", "Secondary Outcome Measures"], inplace=True)

# Display results
from IPython.display import display
display(df_transformed)
print("\n🔹 Top 10 Selected Features:", selected_feature_names[:10])
print("\n✅ Text Preprocessing Complete - Features now show source column.")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yuvik\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yuvik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yuvik\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,NCT Number,Study Title,Study Status,Brief Summary,Study Results,Conditions,Interventions,Sex,Phases,Enrollment,...,TFIDF_secondaryoutcomes_vaccine,TFIDF_secondaryoutcomes_ventilation,TFIDF_secondaryoutcomes_version,TFIDF_secondaryoutcomes_whichever,TFIDF_secondaryoutcomes_woman,TFIDF_secondaryoutcomes_wound,TFIDF_secondaryoutcomes_year,TFIDF_secondaryoutcomes_yearsoverall,TFIDF_secondaryoutcomes_yearstime,TFIDF_secondaryoutcomes_yr
0,NCT00559130,Efficacy Study of CytoSorb Hemoperfusion Devic...,1,The hypothesis of this study is use of CytoSor...,0,0.787961,1.000000,0,7,4.615121,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,NCT00937664,Safety and Tolerability Study of AZD7762 in Co...,0,The primary purpose of this study is to find o...,0,0.871238,0.427615,0,1,3.218876,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,NCT00441597,Does Atorvastatin Reduce Ischemia-Reperfusion ...,1,To study the impact of 3 day exposure to atorv...,0,0.902222,0.916667,2,6,3.433987,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,NCT03296228,Comparison of Dynamic Radiographs in Determini...,1,The purpose of this study is to identify the f...,0,0.826087,0.855230,0,7,4.905275,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,NCT00421603,A Placebo-Controlled Study of Mixed Amphetamin...,1,"The proposed protocol is a double-blind, place...",1,0.858974,0.827615,0,3,4.406719,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257571,NCT02360800,Trial Comparing Tisseel as an Adjunct to Tradi...,1,Bleeding after redo cardiac surgery is a commo...,0,0.900000,0.855230,0,7,4.795791,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
257572,NCT02352506,Serum MIF in Acute Kidney Injury,1,Acute kidney injury (AKI) is a common complica...,0,0.849711,1.000000,0,7,5.872118,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
257573,NCT04996381,Feasibility of AI-based Heart Function Predict...,1,The investigators will develop an artificial i...,0,1.000000,1.000000,0,7,5.872118,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
257574,NCT00380640,The Efficacy of Trimethoprim in Wound Healing ...,1,The purpose of this study is to assess the eff...,0,0.714286,0.855230,0,3,2.397895,...,0.0,0.0,0.0,0.0,0.0,0.238289,0.0,0.0,0.0,0.0



🔹 Top 10 Selected Features: ['TFIDF_primaryoutcomes_abnormality', 'TFIDF_primaryoutcomes_absorption', 'TFIDF_primaryoutcomes_abstinence', 'TFIDF_primaryoutcomes_achieve', 'TFIDF_primaryoutcomes_advanced', 'TFIDF_primaryoutcomes_adverse', 'TFIDF_primaryoutcomes_aes', 'TFIDF_primaryoutcomes_agent', 'TFIDF_primaryoutcomes_alive', 'TFIDF_primaryoutcomes_analogue']

✅ Text Preprocessing Complete - Features now show source column.


819

In [19]:
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from scipy import sparse

# Initialize lemmatizer and stop words
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Separate TF-IDF for each text column
criteria_tfidf = TfidfVectorizer(max_features=1700, ngram_range=(1,1))
summary_tfidf = TfidfVectorizer(max_features=1700, ngram_range=(1,1))
title_tfidf = TfidfVectorizer(max_features=1600, ngram_range=(1,1))

# Fit and transform each column separately
X_criteria = criteria_tfidf.fit_transform(df_transformed["criteria"].fillna("").apply(preprocess_text))
X_summary = summary_tfidf.fit_transform(df_transformed["Brief Summary"].fillna("").apply(preprocess_text))
X_title = title_tfidf.fit_transform(df_transformed["Study Title"].fillna("").apply(preprocess_text))

# Get target variable
y = df_transformed["Study Status"]

# Combine features for chi-square selection
X_combined = sparse.hstack([X_criteria, X_summary, X_title])
combined_features = (
    [f"TFIDF_criteria_{w}" for w in criteria_tfidf.get_feature_names_out()] +
    [f"TFIDF_summary_{w}" for w in summary_tfidf.get_feature_names_out()] +
    [f"TFIDF_title_{w}" for w in title_tfidf.get_feature_names_out()]
)

# Select top features
selector = SelectKBest(chi2, k=500)
X_combined_selected = selector.fit_transform(X_combined, y)

# Get selected feature names with their source
selected_feature_names = [combined_features[i] for i in selector.get_support(indices=True)]

# Convert to DataFrame
tfidf_selected_df = pd.DataFrame(
    X_combined_selected.toarray(),
    columns=selected_feature_names
)

# Merge and clean up
df_transformed = pd.concat([df_transformed.reset_index(drop=True), tfidf_selected_df], axis=1)
df_transformed.drop(columns=["criteria", "Brief Summary", "Study Title"], inplace=True)

# Display results
from IPython.display import display
display(df_transformed)
print("\n🔹 Top 10 Selected Features:", selected_feature_names[:10])
print("\n✅ Text Preprocessing Complete - Features now show source column.")

,NCT Number,Study Status,Study Results,Conditions,Interventions,Sex,Phases,Enrollment,Funder Type,Study Type,...,TFIDF_title_training,TFIDF_title_transplant,TFIDF_title_transplantation,TFIDF_title_treating,TFIDF_title_treatment,TFIDF_title_tumor,TFIDF_title_unresectable,TFIDF_title_vaccine,TFIDF_title_volunteer,TFIDF_title_wound
0,NCT00559130,1,0,0.787961,1.000000,0,7,4.615121,3,0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
1,NCT00937664,0,0,0.871238,0.427615,0,1,3.218876,3,0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
2,NCT00441597,1,0,0.902222,0.916667,2,6,3.433987,6,0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
3,NCT03296228,1,0,0.826087,0.855230,0,7,4.905275,6,1,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
4,NCT00421603,1,1,0.858974,0.827615,0,3,4.406719,6,0,...,0.0,0.0,0.0,0.0,0.21834,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257571,NCT02360800,1,0,0.900000,0.855230,0,7,4.795791,6,0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
257572,NCT02352506,1,0,0.849711,1.000000,0,7,5.872118,6,1,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
257573,NCT04996381,1,0,1.000000,1.000000,0,7,5.872118,6,1,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.00000
257574,NCT00380640,1,0,0.714286,0.855230,0,3,2.397895,6,0,...,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.62494



🔹 Top 10 Selected Features: ['TFIDF_criteria_ablation', 'TFIDF_criteria_absolute', 'TFIDF_criteria_active', 'TFIDF_criteria_adenocarcinoma', 'TFIDF_criteria_adequate', 'TFIDF_criteria_adequately', 'TFIDF_criteria_advanced', 'TFIDF_criteria_adverse', 'TFIDF_criteria_aged', 'TFIDF_criteria_agent']

✅ Text Preprocessing Complete - Features now show source column.


In [14]:
print(df_transformed["Study Status"].isnull().sum())  # Count NaN values

0


In [15]:
df_transformed.to_csv("encoded_data_final_tf_idftarget.csv")

In [22]:
df_transformed.to_csv("encoded_data_final_tf_idftarget1.csv")

DECODE WHEN NEEDED

In [ ]:
# Mapping back encoded values to original categories
df_transformed["Condition (Decoded)"] = df_transformed["Condition"].map(condition_target_map)
df_transformed["Intervention (Decoded)"] = df_transformed["Intervention"].map(intervention_target_map)

# Display a few rows
df_transformed[["Condition (Decoded)", "Condition", "Study Status"]].head()


In [ ]:
# Re-import necessary libraries after execution state reset
import pandas as pd

# Placeholder: Load your dataset again if necessary
# df = pd.read_csv("your_dataset.csv") # Uncomment and replace with actual file

# Dummy data to simulate the encoded dataset
df_target_encoded = pd.DataFrame({
    "Conditions": [0.66, 0.50, 0.58, 0.72, 0.45],  # Encoded values
    "Interventions": [0.60, 0.55, 0.52, 0.65, 0.48]  # Encoded values
})

# Simulated original category mappings (normally these come from your target encoding step)
condition_target_map = {
    "Cancer": 0.66,
    "Diabetes": 0.50,
    "Heart Disease": 0.72,
    "Lung Cancer": 0.45
}

intervention_target_map = {
    "Drug A": 0.60,
    "Drug B": 0.55,
    "Therapy C": 0.52,
    "Device X": 0.65,
    "Drug D": 0.48
}

# Reverse mapping dictionary (encoded value to original category)
reverse_condition_map = {v: k for k, v in condition_target_map.items()}
reverse_intervention_map = {v: k for k, v in intervention_target_map.items()}

# Function to map back encoded values to closest original category
def decode_target_value(value, reverse_map):
    if pd.isna(value):  # If NaN, return NaN
        return None
    closest_match = min(reverse_map.keys(), key=lambda x: abs(x - value))  # Find closest match
    return reverse_map[closest_match]

# Apply decoding
df_target_encoded["Decoded_Conditions"] = df_target_encoded["Conditions"].apply(lambda x: decode_target_value(x, reverse_condition_map))
df_target_encoded["Decoded_Interventions"] = df_target_encoded["Interventions"].apply(lambda x: decode_target_value(x, reverse_intervention_map))

# Display the decoded dataset
import ace_tools as tools
tools.display_dataframe_to_user(name="Decoded Target Encoding", dataframe=df_target_encoded)

# Print a few sample rows
df_target_encoded.head()
